In [1]:
import graphlab

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to ayushjain7398@gmail.com and will expire on June 23, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Ayushj\AppData\Local\Temp\graphlab_server_1530522130.log.0


In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.466565     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.497648     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [19]:
d=model_all['coefficients']

In [21]:
d1=d[d['value']>0]

In [24]:
d1.print_rows(num_rows=20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[6 rows x 4 columns]



In [26]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [43]:
import numpy as np
v={}
for l1_penalty in np.logspace(1,7,num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    pred=model.predict(validation)
    error=validation['price']-pred
    Rs=error**2
    RSS=sum(Rs)
    v[l1_penalty]=RSS
print(min(v.items(),key=lambda x: x[1]))

(10.0, 625766285142461.2)


In [44]:
s=model["coefficients"][model['coefficients']['value']>0]
print model['coefficients']['value'].nnz()
s.print_rows(num_rows=20)

18
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 19604.7163508 |  None  |
|     bedrooms     |  None | 7922.62023075 |  None  |
| bedrooms_square  |  None | 928.112147889 |  None  |
|    bathrooms     |  None |  25392.743443 |  None  |
|   sqft_living    |  None | 39.1083048767 |  None  |
| sqft_living_sqrt |  None | 1123.95233925 |  None  |
|  sqft_lot_sqrt   |  None | 147.859054391 |  None  |
|      floors      |  None |  21154.353574 |  None  |
|  floors_square   |  None |  12871.042877 |  None  |
|    waterfront    |  None | 598538.378522 |  None  |
|       view       |  None | 93089.3048849 |  None  |
|    condition     |  None | 6584.64358167 |  None  |
|      grade       |  None | 6200.37319739 |  None  |
|    sqft_above    |  None | 43.2627983054 |  None  |
|  sqft_basement   |  None | 122.027231745 |  None  |
|     yr_built     |  Non

In [45]:
max_nonzeros = 7

In [46]:
l1_penalty_values = np.logspace(8, 10, num=20)

In [53]:
import pprint
v1={}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    v1[l1_penalty]=model['coefficients']['value'].nnz()

pprint.pprint(v1)    

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313128: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}


In [54]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536


In [57]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
l1_penalty_values

array([  2.97635144e+09,   3.01931664e+09,   3.06228184e+09,
         3.10524703e+09,   3.14821223e+09,   3.19117743e+09,
         3.23414263e+09,   3.27710782e+09,   3.32007302e+09,
         3.36303822e+09,   3.40600341e+09,   3.44896861e+09,
         3.49193381e+09,   3.53489901e+09,   3.57786420e+09,
         3.62082940e+09,   3.66379460e+09,   3.70675980e+09,
         3.74972499e+09,   3.79269019e+09])

In [63]:
v2={}
for l1_penalty in l1_penalty_values:
    model_new=graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    pred1=model_new.predict(validation)
    error=validation['price']-pred1
    rss=sum(error**2)
    v2[l1_penalty]=rss,model_new['coefficients']['value'].nnz()
    
pprint.pprint(v2)

{2976351441.6313128: (966925692362086.8, 10),
 3019316638.952415: (974019450084557.2, 10),
 3062281836.2735171: (981188367942450.9, 10),
 3105247033.5946193: (989328342459472.0, 10),
 3148212230.9157214: (998783211265885.9, 10),
 3191177428.2368236: (1008477167020096.5, 10),
 3234142625.5579257: (1018298780553818.6, 10),
 3277107822.8790278: (1028247992205976.1, 10),
 3320073020.20013: (1034616909232829.2, 8),
 3363038217.5212321: (1038554735941039.9, 8),
 3406003414.8423343: (1043237237871704.1, 8),
 3448968612.1634364: (1046937488751713.6, 7),
 3491933809.4845386: (1051147625612863.0, 7),
 3534899006.8056407: (1055992735342999.2, 7),
 3577864204.1267428: (1060799531763290.2, 7),
 3620829401.447845: (1065707689498229.6, 6),
 3663794598.7689471: (1069464335425583.9, 6),
 3706759796.0900493: (1073504549585594.6, 6),
 3749724993.4111514: (1077632775581412.0, 6),
 3792690190.7322536: (1081867592324111.9, 6)}


In [64]:
bestRSS=1081867592324111.9
for k,v in v2.iteritems():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        bestl1 = k
        
print bestRSS, bestl1

1.04693748875e+15 3448968612.16


In [67]:
model11 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)
n1=model11['coefficients'][model11['coefficients']['value']>0]
n1.print_rows(num_rows=20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

